# Patient selection

In [ ]:
import pandas as pd
import re
import numpy as np

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/trauma_categories_Rega Pain Study15.09.2025_v2.xlsx'
medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/rega_data/rega_physician_list_09102025.xlsx'
meta_medic_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-UniversitédeGenève/icu_research/prehospital/analgesia/data/medreg_extraction/joined_final_complete_extractions_20251008_221735.xlsx'


In [ ]:
restrict_to_secondary = True

In [ ]:
data_df = pd.read_excel(data_path)
medic_df = pd.read_excel(medic_data_path)
meta_medic_df = pd.read_excel(meta_medic_data_path)

medic_df['full_name'] = medic_df['Mitglieder mit Einsatzfunktion'].str.replace(' (Flugarzt/Flugärztin)', '')
medic_df.drop_duplicates(subset=['Mitglieder mit Einsatzfunktion'], inplace=True)
medic_df = medic_df.merge(meta_medic_df, how='left', on='full_name')
medic_df.rename(columns={'Sex m/w': 'physician_sex'}, inplace=True)
data_df = data_df.merge(medic_df, how='left', left_on='Mitglieder mit Einsatzfunktion', right_on='Mitglieder mit Einsatzfunktion')

duplicates = data_df[data_df['SNZ Ereignis Nr. '].duplicated()]['SNZ Ereignis Nr. ']
print(f'Duplicates found: {duplicates.nunique()}')
data_df = data_df.drop_duplicates(subset=['SNZ Ereignis Nr. '])

n_vas_under4 = data_df[data_df['VAS_on_scene'] <= 3].shape[0]
print(f'Excluded {n_vas_under4} patients with VAS <= 3')
data_df = data_df[data_df['VAS_on_scene'] > 3]

n_missing_arrival = data_df['VAS_on_arrival'].isna().sum()
print(f'Excluded {n_missing_arrival} patients with missing VAS_on_arrival')
data_df = data_df.dropna(subset=['VAS_on_arrival'])




In [ ]:
data_df['Einsatzart'].value_counts()

In [ ]:
if restrict_to_secondary:
    n_primary= data_df[data_df['Einsatzart'] != 'Sekundär'].shape[0]
    print(f'Excluded {n_primary} primary transport patients')
    data_df = data_df[data_df['Einsatzart'] == 'Sekundär']